In [1]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 16.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.2
    Uninstalling scipy-1.16.2:
      Successfully uninstalled scipy-1.16.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatib

In [1]:
from gensim.models import Word2Vec
import gensim.downloader
import pandas as pd
import numpy as np
import nltk
import spacy
nltk.download('all')
nlp= spacy.load('en_core_web_sm')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |  

In [8]:
from google.colab import files
uploaded=files.upload()
df= pd.read_csv('netfilx_datasets.csv')
df.shape

Saving netfilx_datasets.csv to netfilx_datasets.csv


(8807, 12)

In [9]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [10]:
df.description

,description
0,"As her father nears the end of his life, filmm..."
1,"After crossing paths at a party, a Cape Town t..."
2,To protect his family from a powerful drug lor...
3,"Feuds, flirtations and toilet talk go down amo..."
4,In a city of coaching centers known to train I...
...,...
8802,"A political cartoonist, a crime reporter and a..."
8803,"While living alone in a spooky town, a young g..."
8804,Looking to survive in a world taken over by zo...
8805,"Dragged from civilian life, a former superhero..."


In [11]:
import warnings
warnings.filterwarnings('ignore')
import re

In [12]:
def data_clean(text):
  text1=' '.join(re.findall('\w+',text))
  doc=nlp(text1)
  clean_text=[token.lemma_ for token in doc if not token.is_stop and not token.is_punct
              and not token.is_digit and not token.is_bracket and not token.is_currency]
  return clean_text

In [13]:
df1= df.description

In [14]:
data=[df1.apply(lambda x: data_clean(x))]

In [15]:
data

[0       [father, near, end, life, filmmaker, Kirsten, ...
 1       [cross, path, party, Cape, Town, teen, set, pr...
 2       [protect, family, powerful, drug, lord, skille...
 3       [Feuds, flirtation, toilet, talk, incarcerated...
 4       [city, coaching, center, know, train, India, s...
                               ...                        
 8802    [political, cartoonist, crime, reporter, pair,...
 8803    [live, spooky, town, young, girl, befriend, mo...
 8804    [look, survive, world, take, zombie, dorky, co...
 8805    [drag, civilian, life, superhero, train, new, ...
 8806    [scrappy, poor, boy, worm, way, tycoon, s, dys...
 Name: description, Length: 8807, dtype: object]

In [16]:
data1= [j for i in data for j in i]
print(data1)

[['father', 'near', 'end', 'life', 'filmmaker', 'Kirsten', 'Johnson', 'stage', 'death', 'inventive', 'comical', 'way', 'help', 'face', 'inevitable'], ['cross', 'path', 'party', 'Cape', 'Town', 'teen', 'set', 'prove', 'private', 'school', 'swimming', 'star', 'sister', 'abduct', 'birth'], ['protect', 'family', 'powerful', 'drug', 'lord', 'skilled', 'thief', 'Mehdi', 'expert', 'team', 'robber', 'pull', 'violent', 'deadly', 'turf', 'war'], ['Feuds', 'flirtation', 'toilet', 'talk', 'incarcerated', 'woman', 'Orleans', 'Justice', 'Center', 'New', 'Orleans', 'gritty', 'reality', 'series'], ['city', 'coaching', 'center', 'know', 'train', 'India', 's', 'fine', 'collegiate', 'mind', 'earnest', 'unexceptional', 'student', 'friend', 'navigate', 'campus', 'life'], ['arrival', 'charismatic', 'young', 'priest', 'bring', 'glorious', 'miracle', 'ominous', 'mystery', 'renew', 'religious', 'fervor', 'die', 'town', 'desperate', 'believe'], ['Equestria', 's', 'divide', 'bright', 'eyed', 'hero', 'believe', '

In [17]:
data2= [j for i in data1 for j in i]
print(data2)

['father', 'near', 'end', 'life', 'filmmaker', 'Kirsten', 'Johnson', 'stage', 'death', 'inventive', 'comical', 'way', 'help', 'face', 'inevitable', 'cross', 'path', 'party', 'Cape', 'Town', 'teen', 'set', 'prove', 'private', 'school', 'swimming', 'star', 'sister', 'abduct', 'birth', 'protect', 'family', 'powerful', 'drug', 'lord', 'skilled', 'thief', 'Mehdi', 'expert', 'team', 'robber', 'pull', 'violent', 'deadly', 'turf', 'war', 'Feuds', 'flirtation', 'toilet', 'talk', 'incarcerated', 'woman', 'Orleans', 'Justice', 'Center', 'New', 'Orleans', 'gritty', 'reality', 'series', 'city', 'coaching', 'center', 'know', 'train', 'India', 's', 'fine', 'collegiate', 'mind', 'earnest', 'unexceptional', 'student', 'friend', 'navigate', 'campus', 'life', 'arrival', 'charismatic', 'young', 'priest', 'bring', 'glorious', 'miracle', 'ominous', 'mystery', 'renew', 'religious', 'fervor', 'die', 'town', 'desperate', 'believe', 'Equestria', 's', 'divide', 'bright', 'eyed', 'hero', 'believe', 'Earth', 'Poni

In [18]:
tokens= [word_tokenize(words) for words in data2]

In [19]:
print(tokens)

[['father'], ['near'], ['end'], ['life'], ['filmmaker'], ['Kirsten'], ['Johnson'], ['stage'], ['death'], ['inventive'], ['comical'], ['way'], ['help'], ['face'], ['inevitable'], ['cross'], ['path'], ['party'], ['Cape'], ['Town'], ['teen'], ['set'], ['prove'], ['private'], ['school'], ['swimming'], ['star'], ['sister'], ['abduct'], ['birth'], ['protect'], ['family'], ['powerful'], ['drug'], ['lord'], ['skilled'], ['thief'], ['Mehdi'], ['expert'], ['team'], ['robber'], ['pull'], ['violent'], ['deadly'], ['turf'], ['war'], ['Feuds'], ['flirtation'], ['toilet'], ['talk'], ['incarcerated'], ['woman'], ['Orleans'], ['Justice'], ['Center'], ['New'], ['Orleans'], ['gritty'], ['reality'], ['series'], ['city'], ['coaching'], ['center'], ['know'], ['train'], ['India'], ['s'], ['fine'], ['collegiate'], ['mind'], ['earnest'], ['unexceptional'], ['student'], ['friend'], ['navigate'], ['campus'], ['life'], ['arrival'], ['charismatic'], ['young'], ['priest'], ['bring'], ['glorious'], ['miracle'], ['om

In [20]:
model= Word2Vec(tokens,min_count=5,vector_size=60,window=3,sg=0)

In [21]:
model.wv.get_vector('bad')

array([-0.01183458,  0.01275853,  0.01362172, -0.01021088,  0.00751159,
        0.0008345 ,  0.00219419,  0.00941009, -0.00742361,  0.00402584,
       -0.01230984,  0.00028886, -0.00858725,  0.0047209 , -0.01355081,
        0.01186621,  0.01249422, -0.00040655,  0.01378035, -0.01517395,
       -0.00479688,  0.01370255,  0.01249922, -0.00558143,  0.0082673 ,
       -0.00847091, -0.00171894,  0.00732333, -0.00176894,  0.00345807,
        0.01022552, -0.00098229, -0.00463026,  0.00818557,  0.0052226 ,
       -0.00188245,  0.00119465, -0.00227023, -0.0116112 , -0.00980263,
        0.01024443,  0.00270128,  0.01616126, -0.0033856 , -0.00774186,
       -0.00947783, -0.01645443,  0.00220362, -0.00564044, -0.0125073 ,
       -0.01355903,  0.00310808,  0.01480308, -0.00076591,  0.00310756,
        0.01070972,  0.00622596, -0.00569389, -0.00031318,  0.0141698 ],
      dtype=float32)

In [22]:
model.wv['bad']

array([-0.01183458,  0.01275853,  0.01362172, -0.01021088,  0.00751159,
        0.0008345 ,  0.00219419,  0.00941009, -0.00742361,  0.00402584,
       -0.01230984,  0.00028886, -0.00858725,  0.0047209 , -0.01355081,
        0.01186621,  0.01249422, -0.00040655,  0.01378035, -0.01517395,
       -0.00479688,  0.01370255,  0.01249922, -0.00558143,  0.0082673 ,
       -0.00847091, -0.00171894,  0.00732333, -0.00176894,  0.00345807,
        0.01022552, -0.00098229, -0.00463026,  0.00818557,  0.0052226 ,
       -0.00188245,  0.00119465, -0.00227023, -0.0116112 , -0.00980263,
        0.01024443,  0.00270128,  0.01616126, -0.0033856 , -0.00774186,
       -0.00947783, -0.01645443,  0.00220362, -0.00564044, -0.0125073 ,
       -0.01355903,  0.00310808,  0.01480308, -0.00076591,  0.00310756,
        0.01070972,  0.00622596, -0.00569389, -0.00031318,  0.0141698 ],
      dtype=float32)

In [23]:
model.wv.most_similar('man')

[('Antonio', 0.44358882308006287),
 ('conflicted', 0.39621537923812866),
 ('Bartleby', 0.3951939344406128),
 ('provoke', 0.3792774975299835),
 ('soviet', 0.3758922219276428),
 ('fed', 0.3737691640853882),
 ('bring', 0.3674168884754181),
 ('viewer', 0.36043989658355713),
 ('piece', 0.3570196032524109),
 ('menace', 0.35522347688674927)]

In [24]:
vec= model.wv['man']+ model.wv['woman']

In [25]:
model.wv.most_similar(vec)

[('woman', 0.6856796145439148),
 ('man', 0.6849305629730225),
 ('Bartleby', 0.48250091075897217),
 ('conflicted', 0.46124711632728577),
 ('attorney', 0.40737318992614746),
 ('raunchy', 0.4031595289707184),
 ('Antonio', 0.3959236145019531),
 ('1980s', 0.3912210762500763),
 ('willing', 0.38815027475357056),
 ('condition', 0.3741723597049713)]

Pre-Trained Models

In [27]:
model= gensim.downloader.info()['models'].keys()

In [28]:
model

dict_keys(['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis'])

In [29]:
model1=gensim.downloader.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [34]:
model1.most_similar('Bengaluru')

[('Bangalore', 0.8312897086143494),
 ('Hyderabad', 0.732100784778595),
 ('Chennai', 0.7134632468223572),
 ('Pune', 0.6997544169425964),
 ('Kolkata', 0.6970940232276917),
 ('Gurgaon', 0.6894409656524658),
 ('Bangaluru', 0.6851993203163147),
 ('Banglore', 0.6755582690238953),
 ('Ahmedabad', 0.6736642122268677),
 ('Mysore', 0.6625763773918152)]

In [36]:
vec= model1['Germany']-model1['Berlin']+model1['paris']

In [37]:
model1.most_similar(vec)

[('paris', 0.8110284209251404),
 ('spain', 0.5597532987594604),
 ('france', 0.5286322236061096),
 ('switzerland', 0.5130846500396729),
 ('germany', 0.5104091763496399),
 ('portugal', 0.5060094594955444),
 ('italy', 0.4973554015159607),
 ('lyon', 0.4966834485530853),
 ('india', 0.493838906288147),
 ('costa_rica', 0.49279069900512695)]

In [38]:
vec= model1['man']+ model1['woman']

In [39]:
model1.most_similar(vec)

[('woman', 0.948119044303894),
 ('man', 0.9308565258979797),
 ('teenage_girl', 0.721599280834198),
 ('girl', 0.7193813323974609),
 ('teenager', 0.6853229403495789),
 ('boy', 0.6776923537254333),
 ('teen_ager', 0.5814452767372131),
 ('lady', 0.5798472762107849),
 ('teenaged_girl', 0.5797287225723267),
 ('person', 0.5755369663238525)]